In [4]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams
import numpy as np

# Connectez-vous à votre instance locale
client = QdrantClient(host="localhost", port=6333)

# Créez une collection avec les paramètres requis
client.recreate_collection(
    collection_name="my_collection",
    vectors_config={
        "default": VectorParams(
            size=128,       # Taille des vecteurs
            distance="Cosine"  # Métrique de distance : Cosine, Euclidean, Dot
        )
    }
)

# Ajoutez des vecteurs
vectors = np.random.rand(10, 128)  # 10 vecteurs aléatoires
ids = list(range(10))  # IDs associés aux vecteurs

client.upload_collection(
    collection_name="my_collection",
    vectors=vectors,
    ids=ids
)

# Effectuez une recherche
query_vector = np.random.rand(128)
results = client.search(
    collection_name="my_collection",
    query_vector=query_vector,
    limit=5
)

print("Résultats de la recherche :", results)

ResponseHandlingException: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams
import numpy as np
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model

# Étape 1 : Modèle de vectorisation
# Chargement du modèle pré-entraîné ResNet50 sans la couche finale
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# Fonction pour extraire les embeddings d'une image
def vectorize_image(image_path):
    # Charger et redimensionner l'image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))

    # Prétraiter l'image pour ResNet
    image = preprocess_input(image.astype('float32'))
    image = np.expand_dims(image, axis=0)

    # Obtenir les embeddings
    vector = model.predict(image)
    return vector.flatten()

# Étape 2 : Initialisation de Qdrant
client = QdrantClient(host="localhost", port=6333)

# Création d'une collection pour les vecteurs d'images
client.recreate_collection(
    collection_name="contacts_faces",
    vectors_config={
        "default": VectorParams(
            size=2048,  # Taille des vecteurs générés par ResNet50
            distance="Cosine"
        )
    }
)

# Étape 3 : Ajout d'un vecteur (exemple avec une image de tête de contact)
image_path = "contact_face.jpg"  # Remplacez par le chemin de votre image
vector = vectorize_image(image_path)

# Ajouter l'image vectorisée à Qdrant
client.upload_collection(
    collection_name="contacts_faces",
    vectors=[vector],  # Le vecteur de l'image
    ids=[1],           # Un ID unique pour cette image
    payload=[{"name": "John Doe"}]  # Métadonnées associées à cette image
)

# Étape 4 : Recherche par similarité
query_image_path = "imageClem.jpg"  # Une autre image pour la recherche
query_vector = vectorize_image(query_image_path)

# Recherche des vecteurs similaires
results = client.search(
    collection_name="contacts_faces",
    query_vector=query_vector,
    limit=5
)

# Afficher les résultats
for result in results:
    print(f"ID: {result.id}, Score: {result.score}, Metadata: {result.payload}")


94765736/94765736 [==============================] - 17s 0us/step


MemoryError: Unable to allocate 9.00 MiB for an array with shape (3, 3, 512, 512) and data type float32